In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import StopWordsRemover
from textblob import TextBlob

import requests
import json

# import nltk
# nltk.download("stopwords")
# stopwordList = nltk.corpus.stopwords.words('portuguese')

In [7]:
if __name__ == "__main__":
    # create Spark session
    spark = SparkSession.builder.appName("TwitterSentimentAnalysis").getOrCreate()
    # read the tweet data from socket
    lines = spark.readStream.format("socket").option("host", "127.0.0.1").option("port", 9999).load()
#     remover = StopWordsRemover(inputCol='word', outputCol="stopWordsRem", stopWords=stopwordList)
    
    words = lines.select(
        explode(
            split(lower(lines.value), " ")
        ).alias('word')
    )
    words = words.filter((words.word == 'buy') | (words.word == 'sell') | (words.word == 'buy') | (words.word == 'bear'))
    
#     words = words.filter(words['word'] == '')
#     words = lines.select(
#         explode(
#             split(
#                 remover.transform(lower(lines.value).alias('word')).select("stopWordsRem").alias('word'),
#                 " ")
#         ).alias("word")
#     )
    wordCounts = words.groupBy("word").count()
    
    query =  wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName("streaming_data") \
    .trigger(processingTime='30 seconds') \
    .start()

In [9]:
df = spark.sql("SELECT * FROM streaming_data")
df.show()

+----+-----+
|word|count|
+----+-----+
| buy|   18|
+----+-----+



In [38]:
remover = StopWordsRemover(inputCol='value', outputCol="words", stopWords=stopwordList)
remover.transform(df)

IllegalArgumentException: requirement failed: Input type must be array<string> but got string.

In [25]:
import time
import pandas as pd
import seaborn as sns

for i in range(10):
    print(len(spark.sql("SELECT * FROM streaming_data").toPandas()))
#     sns.barplot(x = 'word', y = 'count', data = pandas_df.iloc[0:5,:]);
    time.sleep(3)

821
821
821
821
821
821
821
821
821
821


In [6]:
query.stop()

In [81]:
df = spark.sql("SELECT * FROM streaming_data").orderBy('count', ascending = False)
df.show()

+--------------------+-----+
|                word|count|
+--------------------+-----+
|            euphoria|   13|
|                  in|    5|
|                  is|    4|
|                    |    4|
|           #euphoria|    4|
|                  de|    4|
|                   o|    4|
|                 que|    4|
|                  ta|    3|
|              parece|    3|
|               maddy|    3|
|                foto|    3|
|          euphoriart|    3|
|                essa|    3|
|         @pwrsphone:|    3|
|https://t.co/dsrk...|    3|
|                 s2,|    3|
|                 ep4|    3|
|                  em|    3|
|             douglas|    3|
+--------------------+-----+
only showing top 20 rows



In [10]:
 lines.writeStream().start()

TypeError: 'DataStreamWriter' object is not callable

IllegalArgumentException: requirement failed: Input type must be array<string> but got string.

In [21]:
s.decode('utf-8')

AttributeError: 'str' object has no attribute 'decode'

In [18]:
url = "https://api.livecoinwatch.com/coins/single"

payload = json.dumps({
  "currency": "USD",
  "code": "SHIB",
  "meta": False
})
headers = {
  'content-type': 'application/json',
  'x-api-key': 'be34977f-9765-404f-ab83-a97678e83da2'
}

response = requests.request("POST", url, headers=headers, data=payload)

response.text

'{"rate":0.00003106824934034432,"volume":4228292276,"cap":18319975045}'

In [38]:
float(response.text.split(',')[0].split(':')[1])

3.106824934034432e-05